In [1]:
import json
import pymongo
import tweepy

with open('consumer_key.txt', 'r') as f:
    consumer_key =  f.read()
f.closed

with open('consumer_secret.txt', 'r') as f:
    consumer_secret = f.read()
f.closed

with open('access_key.txt', 'r') as f:
    access_key = f.read()
f.closed

with open('access_secret.txt', 'r') as f:
     access_secret = f.read()
f.closed

# Authentication
auth = tweepy.OAuthHandler(consumer_key[:-1], consumer_secret[:-1])
auth.set_access_token(access_key[:-1], access_secret[:-1])
api = tweepy.API(auth)

In [2]:
# Test access

USER_NAME = "nurietta_17"
user = api.get_user(id=USER_NAME)
print (user.description)

VALAR MORGHULIS


In [3]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [5]:
# Find 100 actual tweets located in Spain to store its user in mlab

collection = db['tweet_users']

search_results =api.search(q='*', geocode='39.3262345,-4.8380649,750km', count=100, lang='es')

print(len(search_results))
for i in search_results:
    document = i._json
    collection.insert_one(document)

100


In [27]:
# Get all the tweets stored in the collection "tweet_users" in mlab

cursor = collection.find()
print(cursor)

# Obtain json 
json_docs = []
for doc in cursor:
    doc['_id'] = str(doc['_id'])
    json_docs.append(doc)

In [28]:
# Data format (uploaded to mlab in "tweet_users" collection)

print(json.dumps([json_docs[0]], sort_keys=True, indent=4, separators=(',',':')))

[
    {
        "_id":"5bd4a79db4c0e32bc7f902b1",
        "contributors":null,
        "coordinates":null,
        "created_at":"Sat Oct 27 17:59:42 +0000 2018",
        "entities":{
            "hashtags":[],
            "symbols":[],
            "urls":[],
            "user_mentions":[
                {
                    "id":1031950877387620358,
                    "id_str":"1031950877387620358",
                    "indices":[
                        0,
                        11
                    ],
                    "name":"spooky yume\ud83d\udd77",
                    "screen_name":"__yumecchi"
                }
            ]
        },
        "favorite_count":0,
        "favorited":false,
        "geo":null,
        "id":1056244057838641152,
        "id_str":"1056244057838641152",
        "in_reply_to_screen_name":"__yumecchi",
        "in_reply_to_status_id":1056234978965626881,
        "in_reply_to_status_id_str":"1056234978965626881",
        "in_reply_to_user_id":103

In [63]:
# Get all the usernames

user_names = []

for doc in json_docs:
    user_names.append(doc['user']['screen_name'])

In [64]:
# Number of unique users found

import numpy as np

user_names = set(user_names)
print(len(user_names))

197


In [83]:
# Get 25 tweets of each user until 30th of September of 2018

import time

MAX_ID = 1046439836939415562 # 30 of September

all_tweets = []

for user in user_names:
    
    try:
        new_tweets = api.user_timeline(screen_name=user, count=25, max_id = MAX_ID)
        
    except tweepy.RateLimitError:
        print('Rate limit error with user: %s' % user)
        time.sleep(120)
        continue
    
    except tweepy.TweepError:
        print('Tweep error with user: %s' % user)
        time.sleep(60)
        continue
        
    print(user)
        
    all_tweets.extend(new_tweets)
    
    time.sleep(60)
   

FCJumilla
prosillon
Meendo98
Tweep error with user: DaniQueAdoras
Margari33803379
olartemiguel
CasualsFerrol
Kanrolas
MaraJes21562946
manuel_de_leon
PP_Nervion
javirechulona
Valentina_gnr
emosidoenganada
PBMarbeMalaga
DDanielgv
GuillermoNigot6
Alba404yu
GrandesMedios
Oskarespi
Olga____o
ElLibreroIlegal
deadzino
David_P_D_C
crisman021
tabarniaBCN
solange20896
Criaturita921
CarmensinBG
baratukin
beginagainfilms
_dafnemonroy
matxalen_saitua
matthew_murdok
fch_campos
crnscg
lextresabogados
TmepFan
Postematico
tablero63
CarlosJRubiofm
SofiaEugenia12
auroralopezx
moiceleste
RAGLBLO
nuriialopez19
cbinnobasket
Alberate
rafaelgil251051
pandotbn
SartoTime
IPacuicultura
Hoy_Libro
ainhoowtever
xavidavinxi
javirevi17
_Nadiiee_
YOUCAS1
ANTONIO_JOSE_RB
spaincorral
antoniadeltoro6
balbis_maria
BorjaMetro
IggySanchez
guiadelnino
XLoganLobeznoX
EuroscepticBCN
Lunmbra_
_andreamanzano
LasTaifas
JabierGS
FImpresionante
HAUPOLD
Malakarenovatio
BsardalloRosa
pintadas1996
farena
Energya_VM
thewildzing
brysloh

In [88]:
# Number of tweets from users located in Spain until 30th of September of 2018

print(len(all_tweets))    

4212


In [91]:
# Get tweet id from a known date

n = api.user_timeline(screen_name='nurietta_17', count=2)
for tweet in n:
    print(tweet._json['id'])
    print(tweet._json['created_at'])

1048919100423118848
Sun Oct 07 12:52:56 +0000 2018
1046439836939415562
Sun Sep 30 16:41:13 +0000 2018


In [89]:
# Store all tweets found in mlab "tweets" collection

collection = db['tweets']

for tweet in all_tweets:
    collection.insert_one(tweet._json)